# Generación de Batería de Preguntas para TFG

Este notebook tiene dos partes:
1. **Exploración de la Base de Datos:** Usamos Elasticsearch con parámetros flexibles para descubrir de qué hablan las noticias indexadas y extraer hechos concretos.
2. **Generación del Dataset:** Creación y guardado del archivo `bateria_preguntas_tfg.csv` con el formato requerido por el Juez LLM.

In [ ]:
from elasticsearch import Elasticsearch

# 1. CONEXIÓN (Túnel SSH)
es = Elasticsearch("http://127.0.0.1:9250")

try:
    if es.ping():
        print(f"Conectado a: {es.info()['name']} (Listo para explorar)")
    else:
        print("El servidor no responde. ¿Está el túnel abierto?")
except Exception as e:
    print(f"Error inesperado: {e}")

# 2. FUNCIÓN DE EXPLORACIÓN (Relajada)
def explorar_noticias(query, k=3): # k=3 para no saturar la pantalla
    print(f"\nEXPLORANDO TEMA: '{query}'")
    print("=" * 80)
    
    search_payload = {
        "size": k,
        "query": {
            "multi_match": {
                "query": query,
                "fields": ["title^3", "body"], 
                "fuzziness": "AUTO", # Tolerancia a errores (solo para explorar)
                "operator": "or"     # Búsqueda amplia
            }
        },
        # IMPORTANTE: Traemos el body para poder leer el dato exacto
        "_source": ["title", "date", "source", "body"] 
    }
    
    try:
        response = es.search(index="noticias_tfg", body=search_payload) 
        hits = response['hits']['hits']
        
        if not hits:
            print("No se encontraron noticias sobre este tema.")
            return

        for i, hit in enumerate(hits):
            data = hit['_source'] 
            clean_date = data.get('date', 'N/A')[:10] if data.get('date') else "Sin fecha"
            
            print(f"{i+1}. TÍTULO: {data.get('title', 'Sin título')}")
            print(f"FECHA:  {clean_date} | FUENTE: {data.get('source', 'Desconocida')}")
            # Imprimimos los primeros 400 caracteres del cuerpo para buscar el "dato"
            cuerpo = data.get('body', '')
            print(f"TEXTO:  {cuerpo}...\n")
            print("-" * 80)
            
    except Exception as e:
        print(f"Error durante la búsqueda: {e}")

Conectado a: valencia (Listo para explorar)


In [16]:
explorar_noticias("guerra comercial Trump aranceles China")

explorar_noticias("inteligencia artificial regulación")

explorar_noticias("fichaje Kylian Mbappe Real Madrid")



EXPLORANDO TEMA: 'guerra comercial Trump aranceles China'
1. TÍTULO: Trump amenaza con más aranceles a móviles en su guerra comercial con China
FECHA:  2025-04-14 | FUENTE: La Razón
TEXTO:  El presidente de Estados Unidos, Donald Trump, ha dicho en las últimas horas que él puede predecir el futuro, luego de que los mercados reaccionaran positivamente el lunes a su decisión de excluir los teléfonos inteligentes y otros productos electrónicos de los aranceles ultraelevados sobre bienes importados desde China. «¡LA MEJOR DEFINICIÓN DE INTELIGENCIA ES LA CAPACIDAD DE PREDECIR EL FUTURO!» publicó el mandatario en su plataforma Truth Social. El republicano ha insistido además en que ningún país está «libre de culpa» por los «desequilibrios comerciales injustos», en vísperas de sus nuevos aranceles esperados sobre potentes chips de computadora, comúnmente conocidos como semiconductores. Trump agregó que esos productos no estarían completamente exentos de aranceles, sino que han sido colocado

In [17]:
explorar_noticias("Pedro Sánchez congreso")

explorar_noticias("Elon Musk aranceles europa")


EXPLORANDO TEMA: 'Pedro Sánchez congreso'
1. TÍTULO: Todos los señalados por Pedro Sánchez en el Congreso Federal del PSOE
FECHA:  2024-12-03 | FUENTE: ABC.es
TEXTO:  Pedro Sánchez aseguró en la clausura el domingo en Sevilla del 41 Congreso Federal del PSOE que su «principal prioridad» sería revalidar su propio mandato en 2027 y, al mismo tiempo, recuperar el poder territorial que perdieron los socialistas frente al PP ... en 2023.Andalucía Juan Espadas De candidato contra Díaz a discutidoEl líder del PSOE andaluz ha pasado en dos años de ser el elegido por Ferraz para derrocar a Susana Díaz en unas primarias a caer en desgracia. Por si quedaba alguna duda, Sánchez la despejó el domingo al evitar ratificarle como candidato, un desprecio aún mayor al producirse cuando presidía el congreso y en la ciudad de la que fue alcalde. Nada más terminar el cónclave, Espadas confirmó que irá a unas primarias y retó a los críticos a presentarse. Noticia Relacionada estandar Si Sánchez apuesta por

In [ ]:
import pandas as pd

# Definimos la batería de preguntas manualmente
datos = [
    {
        "id": 1,
        "tipo": "Factual",
        "pregunta": "¿Qué porcentaje total de aranceles impuso Trump a China en abril de 2025?",
        "respuesta_esperada": "145% (125% nuevo más 20% existente)."
    },
    {
        "id": 2,
        "tipo": "Factual",
        "pregunta": "¿Cuánto dinero perdió Elon Musk tras el anuncio de los aranceles?",
        "respuesta_esperada": "17.800 millones de dólares en dos días."
    },
    
    {
        "id": 3,
        "tipo": "Trampa",
        "pregunta": "¿Qué opinó el Rey de España sobre los aranceles de Trump a China?",
        "respuesta_esperada": "No tengo información suficiente en mis archivos."
    },
     {
        "id": 4,
        "tipo": "Trampa",
        "pregunta": "¿Qué dijo Elon Musk sobre los aranceles a China?",
        "respuesta_esperada": "No tengo información suficiente en mis archivos."
    },

    {
        "id": 5,
        "tipo": "Inferencia",
        "pregunta": "¿Qué relación quiere Elon Musk entre Estados Unidos y Europa según sus declaraciones en Italia?",
        "respuesta_esperada": "Quiere una zona de libre comercio con aranceles cero."
    },

    {
        "id": 6,
        "tipo": "Factual",
        "pregunta": "¿Qué presidente firmó un decreto para centralizar la regulación de la inteligencia artificial?",
        "respuesta_esperada": "Donald Trump."
    },

    {
        "id": 7,
        "tipo": "Inferencia",
        "pregunta": "¿Por qué Donald Trump no quiere que los estados regulen la inteligencia artificial por su cuenta?",
        "respuesta_esperada": "Porque cree que la IA estadounidense no tendrá éxito si hay 50 fuentes distintas de aprobación, argumentando que debe haber un único marco legal nacional para ganar la carrera tecnológica a China."
    },

    {
        "id": 8,
        "tipo": "Trampa",
        "pregunta": "¿Qué opinó Felipe González sobre el decreto de inteligencia artificial de Trump?",
        "respuesta_esperada": "No tengo información suficiente en mis archivos."
    },

    {
        "id": 9,
        "tipo": "Factual",
        "pregunta": "¿Por cuántas temporadas ficha Kylian Mbappé con el Real Madrid?",
        "respuesta_esperada": "Por cinco temporadas."
    },

    {
        "id": 10,
        "tipo": "Inferencia",
        "pregunta": "¿Por qué el artículo de El País dice que el comunicado del fichaje de Mbappé cayó en medio de una resaca emocional?",
        "respuesta_esperada": "Porque el anuncio se hizo menos de 24 horas después de que terminaran las celebraciones por la 15ª Copa de Europa del Real Madrid."
    },

    {
        "id": 11,
        "tipo": "Factual",
        "pregunta": "¿Cuál dijo Pedro Sánchez que era su principal prioridad en la clausura del 41 Congreso Federal del PSOE?",
        "respuesta_esperada": "Revalidar su propio mandato en 2027 y recuperar el poder territorial perdido frente al PP en 2023."
    },

    {
        "id": 12,
        "tipo": "Trampa", # La noticia habla de un lapsus, pero no de su dimisión.
        "pregunta": "¿En qué fecha dimitió Pedro Sánchez tras su lapsus en el Congreso de los Diputados?",
        "respuesta_esperada": "No tengo información suficiente en mis archivos."
    },

    {
        "id": 13,
        "tipo": "Factual",
        "pregunta": "¿Qué proponen Elon Musk y Sam Altman para redistribuir la riqueza algorítmica?",
        "respuesta_esperada": "Proponen repartir dinero para todos sin necesidad de trabajar (renta básica universal) mediante el uso de tokens digitales."
    },
    {
        "id": 14,
        "tipo": "Inferencia",
        "pregunta": "¿Por qué fue demandado Elon Musk por la Fiscalía de Filadelfia en 2024?",
        "respuesta_esperada": "Porque prometió dar 1 millón de dólares diarios a personas seleccionadas al azar que firmaran un documento apoyando la libertad de expresión y el derecho a las armas, con el objetivo de impulsar la campaña electoral de Trump."
    },

    {
        "id": 15,
        "tipo": "Trampa", 
        "pregunta": "¿Cuántos tokens digitales propuso repartir Donald Trump como renta básica universal?",
        "respuesta_esperada": "No tengo información suficiente en mis archivos."
    }
]

# Convertimos a DataFrame
df_bateria = pd.DataFrame(datos)

# Mostramos una vista previa
print(f"✅ Batería creada con {len(df_bateria)} preguntas.")
display(df_bateria)

✅ Batería creada con 15 preguntas.


,id,tipo,pregunta,respuesta_esperada
0,1,Factual,¿Qué porcentaje total de aranceles impuso Trum...,145% (125% nuevo más 20% existente).
1,2,Factual,¿Cuánto dinero perdió Elon Musk tras el anunci...,17.800 millones de dólares en dos días.
2,3,Trampa,¿Qué opinó el Rey de España sobre los arancele...,No tengo información suficiente en mis archivos.
3,4,Trampa,¿Qué dijo Elon Musk sobre los aranceles a China?,No tengo información suficiente (la noticia ha...
4,5,Inferencia,¿Qué relación quiere Elon Musk entre Estados U...,Quiere una zona de libre comercio con arancele...
5,6,Factual,¿Qué presidente firmó un decreto para centrali...,Donald Trump.
6,7,Inferencia,¿Por qué Donald Trump no quiere que los estado...,Porque cree que la IA estadounidense no tendrá...
7,8,Trampa,¿Qué opinó Sam Altman sobre el decreto de inte...,No tengo información suficiente en mis archivos.
8,9,Factual,¿Por cuántas temporadas firmó Kylian Mbappé co...,Por cinco temporadas.
9,10,Inferencia,¿Por qué el artículo de El País dice que el co...,Porque el anuncio se hizo menos de 24 horas de...
